In [1]:
import numpy as np
import pandas as pd
import sqlite3
import glob


# Set up logger
from logging import getLogger, StreamHandler, Formatter, INFO

logger = getLogger(__name__)
format = Formatter(fmt='%(asctime)s [%(levelname)s] - %(message)s',
                   datefmt='%Y-%m-%d %H:%M:%S')
handler = StreamHandler()
handler.setLevel(INFO)
handler.setFormatter(format)
logger.addHandler(handler)
logger.setLevel(INFO)
logger.propagate = False

In [21]:
# Columns to use
usecols_new = ['Date', 'YCU ID', 'DNA ID', '病名', 'State', 'Gene', 'WES batch', 
               '家族関係', 'father DNA ID', 'mother DNA ID', 'Proband_DNA_ID']

usecols_old = ['Date', 'YCU ID', 'DNA ID', '病名', 'WES batch', 
               '家族関係', 'father DNA ID', 'mother DNA ID', 'Proband_DNA_ID']

usecols_post = ['検体拝受日', 'YCUID', 'DNAID', '診断名', 'State', 
                '解析結果(遺伝子名)', 'ProbandID']

## Find the old excel file and the latest excel file
logger.info("Find the excel filed")
old_excel = glob.glob('rawdata/2004*.xlsx')[0]
new_excel = glob.glob('rawdata/自動報告書*')[0]
post_excel = glob.glob('rawdata/郵送*')[0]

# Load excel files as pd.DataFrame
logger.info(f"Load {new_excel} file")
df = pd.read_excel(new_excel, dtype=str, sheet_name='database', 
                   index_col=None, skiprows=3, usecols=usecols_new)
df.dropna(how='all')
df = df[usecols_new]

logger.info(f"Load {old_excel} file")
df_old = pd.read_excel(old_excel, index_col=None, usecols=usecols_old)
df_old.dropna(how='all')
df_old = df_old[usecols_old]

logger.info(f"Load {post_excel} file")
df_post = pd.read_excel(post_excel, index_col=None, usecols=usecols_post)
df_post.dropna(how='all')
df_post = df_post[usecols_post]

# Pre-porcessing for State information
replace_to_identified = [
    '89717692', '22006383', '166210771', '18602433', '18598085', '53279465', 
    '6495539', '89653823', '89712015', '37840968', '47990943', '24024725', 
    '39929311', '29,610,505', '50607726', '50607674', '75959244', '220154743', 
    '220156581', '56385389', '8242896', '166909430', '153363092', '166237229',
    '62070967', '105834449', '130422388', '1736004', '52082841', '101953473',
    '101953473', 'Confirm', 'SNV_identified', 'CNV_identified', 'SV_identified', 
    'SV_determined', 'identified', 'Undetermined→identified', 
    'Repeat_expansion', 'repeat expansion identified', 'mosicism susp',
    'SNV_identified;CNV_identified', 'Tandem_repeat_expansion_identified'
    ]

replace_to_undetermined = [
    'On going', 'On_going', 'undetermined', '検体取り下げ', '欠番', 'Unknown',
    'Inconclusive', 'サンガーの結果、本家系で見られたCOL4A2バリアントはなかった'
    ]

df.replace(replace_to_identified, 'Identified', inplace=True)
df.replace(replace_to_undetermined, 'Undetermined', inplace=True)
df.fillna('Undetermined', inplace=True)
df_post.replace(replace_to_identified, 'Identified', inplace=True)
df_post.replace(replace_to_undetermined, 'Undetermined', inplace=True)
df_post.fillna('Undetermined', inplace=True)

if len(df['State'].unique()) > 2:
    errmsg = f"State column has more than 2 unique values. In the {new_excel}."
    logger.error(errmsg)
    exit(1)

if len(df_post['State'].unique()) > 2:
    errmsg = f"State column has more than 2 unique values. In the {post_excel}."
    logger.error(errmsg)
    exit(1)

# Rename columns
rename_dict = {'YCU ID': 'YCU_ID', 'YCUID': 'YCU_ID', 
               'DNA ID': 'DNA_ID', 'DNAID': 'DNA_ID', 
               }

df.rename(columns=rename_dict, inplace=True)
df_old.rename(columns=rename_dict, inplace=True)
df_post.rename(columns=rename_dict, inplace=True)

# Connect to sqlite3 database
ycu_db = 'db/ycudb.db'
logger.info("Connect to sqlite3 database")
conn = sqlite3.connect(ycu_db)

# Create sqlite database (If exists, replace). Three tables will be created.
logger.info("Create sqlite database")
df.to_sql('new_samples', conn, if_exists='replace', index=False)
df_old.to_sql('old_samples', conn, if_exists='replace', index=False)
df_post.to_sql('post_samples', conn, if_exists='replace', index=False)

# Close connection
conn.close()


2024-01-04 18:20:14 [INFO] - Find the excel filed
2024-01-04 18:20:14 [INFO] - Load rawdata/自動報告書作成_データベース20231228.xlsm file


In [13]:
conn = sqlite3.connect(ycu_db)

In [17]:
ycu_id = '231694'
query = "SELECT YCU_ID,State FROM new_samples WHERE `YCU_ID` = ?"
cursor = conn.cursor()
cursor.execute(query, (ycu_id,))

results = cursor.fetchall()
for row in results:
    print(row)


OperationalError: no such column: YCU_ID